In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
import os
import random
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

In [ ]:
def set_all_seeds(seed):
    os.environ["PL_GLOBAL_SEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    tf.experimental.numpy.random.seed(seed)
    os.environ['TF_CUDNN_DETERMINISTIC'] = '1'
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
    os.environ["PYTHONHASHSEED"] = str(seed)

set_all_seeds(420)

In [ ]:
model_mn2 = MobileNetV2(
    input_shape=None, alpha=1.0, include_top=True, weights='imagenet',
    input_tensor=None, pooling=None, classes=1000
)

14536120/14536120 [==============================] - 0s 0us/step


In [ ]:
model_mn2.summary()

In [ ]:
layer_name = 'block_15_add'
feature_extractor = Model(inputs=model_mn2.input, outputs=model_mn2.get_layer(layer_name).output)

In [ ]:
plants_names = []

for plant in os.listdir('gdrive/MyDrive/flower_data/encoded/train'):
    plants_names += [plant.split('.npy')[0]]

plants_names.sort()
plant_name_to_id = {plant: i for i, plant in enumerate(plants_names)}

In [ ]:
instances = {}
labels = {}

for set_kind in ["test", "train", "val"]:
    X_per_class = []
    y_per_class = []

    for i, file in enumerate(os.listdir(f"gdrive/MyDrive/flower_data/encoded/{set_kind}")):
        print(f"\r[{i}] {set_kind}/{file}", end='')
        plants_encoded = np.load(f"gdrive/MyDrive/flower_data/encoded/{set_kind}/{file}")
        plant_name = file.split('.npy')[0]

        X_per_class += [plants_encoded.reshape([plants_encoded.shape[0], 49, 160])]
        y_per_class += [plant_name_to_id[plant_name] for _ in range(plants_encoded.shape[0])]

    instances[set_kind] = np.concatenate(X_per_class)
    labels[set_kind] = np.array(y_per_class)

[101] val/16.npy

In [ ]:
n_classes = len(plants_names)

X_train = instances['train']
y_train = np.eye(n_classes)[labels['train']]

X_valid = instances['val']
y_valid = np.eye(n_classes)[labels['val']]

X_test = instances['test']
y_test = np.eye(n_classes)[labels['test']]

In [ ]:
pca_per_filter = [
    PCA(n_components=5).fit(X_train[..., i])
    for i in range(X_train.shape[-1])
]

In [ ]:
all_means = np.array([filter.mean_ for filter in pca_per_filter])
all_means = all_means.T.reshape([1, 49, 160])

all_components = np.array([filter.components_ for filter in pca_per_filter])
all_components = np.transpose(all_components, [2, 1, 0])

filterwise_dot = np.einsum('ij...,jk...->...ik', X_train - all_means, all_components)
filterwise_dot = np.transpose(filterwise_dot, axes=[1, 2, 0])

X_train_pca = filterwise_dot

In [ ]:
all_means = np.array([filter.mean_ for filter in pca_per_filter])
all_means = all_means.T.reshape([1, 49, 160])

all_components = np.array([filter.components_ for filter in pca_per_filter])
all_components = np.transpose(all_components, [2, 1, 0])

filterwise_dot = np.einsum('ij...,jk...->...ik', X_valid - all_means, all_components)
# equivalent to einsum tf.stack([A[..., i] @ B[..., i] for i in tf.range(160)], axis=-1)
filterwise_dot = np.transpose(filterwise_dot, axes=[1, 2, 0])

X_valid_pca = filterwise_dot

In [ ]:
all_means = np.array([filter.mean_ for filter in pca_per_filter])
all_means = all_means.T.reshape([1, 49, 160])

all_components = np.array([filter.components_ for filter in pca_per_filter])
all_components = np.transpose(all_components, [2, 1, 0])

filterwise_dot = np.einsum('ij...,jk...->...ik', X_test - all_means, all_components)
# equivalent to einsum tf.stack([A[..., i] @ B[..., i] for i in tf.range(160)], axis=-1)
filterwise_dot = np.transpose(filterwise_dot, axes=[1, 2, 0])

X_test = filterwise_dot

In [ ]:
model = keras.models.Sequential([
    keras.layers.Input(shape=[5, 160]),
    keras.layers.BatchNormalization(),
    keras.layers.SeparableConv1D(filters=320, depth_multiplier=3, kernel_size=5),
    keras.layers.ELU(),
    keras.layers.Flatten(),
    keras.layers.Dropout(rate=0.15),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(480, kernel_initializer="lecun_normal", use_bias=False),
    keras.layers.ELU(),
    keras.layers.Dropout(rate=0.25),
    keras.layers.Dense(102, activation="softmax")
])

In [ ]:
model.compile(
    loss="categorical_crossentropy",
    optimizer="nadam",
    metrics=["accuracy"]
)

In [ ]:
model.summary()

In [ ]:
idx = np.arange(X_train_pca.shape[0])
np.random.shuffle(idx)

X_train_pca_shuffled = X_train_pca[idx]
y_train_shuffled = y_train[idx]

In [ ]:
lr_scheduler = keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=2)

model.fit(
    X_train_pca_shuffled, y_train_shuffled,
    epochs=10,
    validation_data=(X_valid_pca, y_valid),
    callbacks=[lr_scheduler]
)

Epoch 1/10
333/333 [==============================] - 16s 8ms/step - loss: 1.2111 - accuracy: 0.7075 - val_loss: 0.5020 - val_accuracy: 0.8782 - lr: 0.0010
Epoch 2/10
333/333 [==============================] - 3s 8ms/step - loss: 0.2879 - accuracy: 0.9182 - val_loss: 0.3304 - val_accuracy: 0.8964 - lr: 0.0010
Epoch 3/10
333/333 [==============================] - 2s 6ms/step - loss: 0.1811 - accuracy: 0.9479 - val_loss: 0.2922 - val_accuracy: 0.9216 - lr: 0.0010
Epoch 4/10
333/333 [==============================] - 2s 6ms/step - loss: 0.1237 - accuracy: 0.9600 - val_loss: 0.3809 - val_accuracy: 0.9020 - lr: 0.0010
Epoch 5/10
333/333 [==============================] - 2s 6ms/step - loss: 0.1279 - accuracy: 0.9608 - val_loss: 0.3819 - val_accuracy: 0.8964 - lr: 0.0010
Epoch 6/10
333/333 [==============================] - 2s 6ms/step - loss: 0.0648 - accuracy: 0.9806 - val_loss: 0.2817 - val_accuracy: 0.9286 - lr: 5.0000e-04
Epoch 7/10
333/333 [==============================] - 2s 7ms/step

In [ ]:
model.save("model_pca_sepconv")
!zip -r model_pca_sepconv.zip model_pca_sepconv/

  adding: model_pca_sepconv/ (stored 0%)
  adding: model_pca_sepconv/fingerprint.pb (stored 0%)
  adding: model_pca_sepconv/variables/ (stored 0%)
  adding: model_pca_sepconv/variables/variables.data-00000-of-00001 (deflated 9%)
  adding: model_pca_sepconv/variables/variables.index (deflated 63%)
  adding: model_pca_sepconv/keras_metadata.pb (deflated 90%)
  adding: model_pca_sepconv/assets/ (stored 0%)
  adding: model_pca_sepconv/saved_model.pb (deflated 88%)
